In [ ]:
# Load and Manipulate Data

In [5]:
from scipy.io import loadmat
import pandas as pd
import numpy as np
import math as math
import random as random
from sklearn import linear_model
train_faces = loadmat('faces.mat')['train_faces']
train_nonfaces = loadmat('faces.mat')['train_nonfaces']
test_faces = loadmat('faces.mat')['test_faces']
test_nonfaces = loadmat('faces.mat')['test_nonfaces']

# Add Y response to training and testing sets
N=2429
zeros = np.zeros((N,1))
ones = np.ones((N,1))
train_faces =  np.append(train_faces, ones, 1)
train_faces = np.uint8(train_faces)
train_nonfaces = np.append(train_nonfaces, zeros, 1)
train_nonfaces = np.uint8(train_nonfaces)

train_faces[0:2]
train_nonfaces[0:3]

N=472
zeros = np.zeros((N,1))
ones = np.ones((N,1))
test_faces =  np.append(test_faces, ones, 1)
test_faces = np.uint8(test_faces)
test_nonfaces = np.append(test_nonfaces, zeros, 1)
test_nonfaces = np.uint8(test_nonfaces)

# merge train_faces with train_nonfaces, test_faces with test_nonfaces
training_data = np.concatenate((train_faces, train_nonfaces), axis=0)
training_data = np.insert(training_data, 0, 1, axis=1)

testing_data = np.concatenate((test_faces, test_nonfaces), axis=0)
testing_data = np.insert(testing_data, 0, 1, axis=1)
print(training_data.shape) 
print(testing_data.shape)

(4858, 363)
(944, 363)


In [ ]:
# Question 1 Part a

In [11]:
# Implement logistic regression
def logistic_func(X,y, num_beta, maxiter):
    beta_start = np.zeros((num_beta))
    beta = beta_start
    i = 0
    diff = np.ones((maxiter))
    J_bar =np.zeros((num_beta,num_beta))
#    l=0
    for i in range(maxiter):
        beta_old = beta
        p = np.exp(np.dot(X, beta))/ (1. + np.exp(np.dot(X, beta)))
#        l = np.sum(y*np.log(p) + (1.-y)*np.log(1.-p)) # log-likeliehood
        s = np.dot(y-p,X).reshape(362,1)                            # scoring function
        J_bar = np.dot(X.T*np.multiply(p,1.-p),X)      # information matrix
        beta = beta_old + np.ravel(np.dot(np.linalg.inv(J_bar),s))# new value of beta
        diff[i] = np.sum(np.fabs(beta-beta_old)) # sum of absolute differences
        i += 1
    
    return beta, J_bar, diff
random.seed(2017)
results = logistic_func(X=training_data[:,0:362], y = training_data[:,-1],maxiter =1000,num_beta = 362)
print("The first 5 parameter estimates are",results[0][0:5]) # First 5 Beta hats


beta = results[0]

# training: predicted y
y_hat_train = np.round(1 / (1 + np.exp(-np.dot(training_data[:,0:362],beta)))) 
# Training Accuracy:0.985
# Training Error: 0.015
print("Training Accuracy is",(y_hat_train == training_data[:,-1]).sum().astype(float) / len(y_hat_train))
print("Training Error is", 1 - ((y_hat_train == training_data[:,-1]).sum().astype(float) / len(y_hat_train)))

# testing: predicted y
y_hat_test = np.round(1 / (1 + np.exp(-np.dot(testing_data[:,0:362],beta)))) 
# Testing Accuracy: 0.519
# Testing Error: 0.481
print("Testing Accuracy is",(y_hat_test == testing_data[:,-1]).sum().astype(float) / len(y_hat_test))
print("Testing Error is", 1 - ((y_hat_test == testing_data[:,-1]).sum().astype(float) / len(y_hat_test)))


The first 5 parameter estimates are [ -1.57669975e+01  -9.86760937e-02  -2.76040412e-02   3.76605690e-02
   2.12393352e-04]
Training Accuracy is 0.985384932071
Training Error is 0.0146150679292
Testing Accuracy is 0.51906779661
Testing Error is 0.48093220339


In [ ]:
# Question 1 Part b

In [7]:
np.random.shuffle(training_data)
X = training_data[:,1:362] / training_data[:,1:362].max(axis=0)
print('train)\tmin: ', np.min(X), '\tmax: ', np.max(X))

#Output
y=training_data[:,-1].reshape(4858,1)

#Sigmoid Function
def sigmoid(x):
    return 1/(1 + np.exp(-x))

#Derivative of Sigmoid Function
def derivatives_sigmoid(x):
    return x * (1 - x)

def NN2(X, y, epoch, lr):
    lr = lr
    NumInputNeurons = 361
    NumHiddenNeurons = 4
    NumOutputNeurons = 1
      
#    wh=np.random.uniform(low=0.0, high=1.0/1000000, size=(NumInputNeurons,NumHiddenNeurons))
#    wout=np.random.uniform(low=0.0, high=1.0/1000000,size=(NumHiddenNeurons,NumOutputNeurons))
    random.seed(2017)
    wh=np.random.uniform(low=-0.5, high=0.5, size=(NumInputNeurons,NumHiddenNeurons))
    wout=np.random.uniform(low=-0.5, high=0.5,size=(NumHiddenNeurons,NumOutputNeurons))
    for i in range(epoch):
        for j in range(4858):
            # feedforward for each observation/row in X
            a_j = np.dot(X[[j]].reshape(1,361), wh) # a_j[j] is 1 by 4 vector: linear comb output in hidden layer based on X[j]= 4 values
            z_j = sigmoid(a_j) # z_j[j] is 1 by 4 vector: sigmoid transformation output in hidden layer based on X[j]
            a_k = np.dot(a_j.reshape(1,4), wout) # a_k[j] is 1 by 1 value in output layer
            z_k = sigmoid(a_k) # z_k[j] is 1 by 1 value in output layer   
               
            # backpropagation
            wout = np.ravel(wout) + (2 * lr *(y[[j]] - z_k) * (z_k * (1-z_k))) * z_j
            wout = wout.reshape(4,1)
            
            a = ((2 * lr *(y[[j]] - z_k) * (z_k * (1-z_k))) * z_j).T # 4 by 1
            b = np.dot(X[[j]].T, (np.ravel(a) * np.ravel(wout) * np.ravel(z_j * (1-z_j))).reshape(1,4))
            wh = wh + b
    return wh, wout

train)	min:  0.0 	max:  1.0


In [8]:
weight = NN2(X,y, epoch = 50, lr = 0.01)    

In [10]:
# predict on training data
random.seed(2017)
np.random.shuffle(training_data)
X_train = training_data[:,1:362] / training_data[:,1:362].max(axis=0)
# print('train)\tmin: ', np.min(X_train), '\tmax: ', np.max(X))

#Output
y_train=training_data[:,-1].reshape(4858,1)
      
a_j = np.dot(X_train, weight[0]) # a_j is 4858 by 4 vector: linear comb output in hidden layer based on X
z_j = sigmoid(a_j) # z_j is 4858 by 4 vector: sigmoid transformation output in hidden layer based on X
a_k = np.dot(a_j, weight[1]) # a_k is 4858 by 1 value in output layer
z_k = sigmoid(a_k)

prediction_training = np.zeros((4858,1))
for i in range(4858):
    if z_k[[i]] >= 0.50:
        prediction_training[[i]] = 1
    else:
        prediction_training[[i]] = 0

print("Training Accuracy is",(np.sum(prediction_training == y_train))/ len(prediction_training)) # training accuracy 0.99
print("Training Error is", 1 - ((np.sum(prediction_training == y_train))/ len(prediction_training)))

# predict on testing data
random.seed(2017)
np.random.shuffle(testing_data)
X_test = testing_data[:,1:362] / testing_data[:,1:362].max(axis=0)
# print('test)\tmin: ', np.min(X_test), '\tmax: ', np.max(X_test))

#Output
y_test=testing_data[:,-1].reshape(944,1)

a_j1 = np.dot(X_test, weight[0]) # a_j1 is 944 by 4 vector: linear comb output in hidden layer based on X
z_j1 = sigmoid(a_j1) # z_j1 is 944 by 4 vector: sigmoid transformation output in hidden layer based on X
a_k1 = np.dot(z_j1, weight[1]) # a_k1 is 944 by 1 value in output layer
z_k1 = sigmoid(a_k1)

prediction_testing = np.zeros((944,1))
for i in range(944):
    if z_k1[[i]] >= 0.0050:
        prediction_testing[[i]] = 1
    else:
        prediction_testing[[i]] = 0
print("Testing Accuracy is",(np.sum(prediction_testing == y_test))/ len(prediction_testing))
print("Testing Error is", 1 - ((np.sum(prediction_testing == y_test))/ len(prediction_testing)))


Training Accuracy is 0.930218196789
Training Error is 0.0697818032112
Testing Accuracy is 0.501059322034
Testing Error is 0.498940677966
